# David Viar

Los pasos que vamos a realizar son los siguientes:

1. Las funciones básicas pedidas por Idoven

    

*   Be able to read the ECG files and corresponding annotations
*   Show how they will work on the signal and plot the signal in appropriate manner to be read by a doctor

*   Identify the heart beat of the signal, average and total heart beat in the signal
*   Identify the complex QRS in the signal and been able to annotate on it

2. Clasificar señales de ecg entre Myocardical infarction y paciente sano, desarrollando un método de clasíficación propio y tomando ventaja de las 15 derivaciones.







    
    
    

In [ ]:
!wget https://physionet.org/static/published-projects/ptbdb/ptb-diagnostic-ecg-database-1.0.0.zip

Descomprimimos el zip

In [ ]:
!unzip ptb-diagnostic-ecg-database-1.0.0.zip

Importamos las librerías necesarias con las que vamos a trabajar

Después de probar con pandas y numpy varias formas de leer los archivos, me dí cuenta que se planteaba dificil. Decidí (como se debería haber hecho desde el principio) leer la guia de la base de datos. Después de saltar por varias webs encontre el modulo wfdb.

In [ ]:
!pip install wfdb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as ps

import wfdb
from wfdb import processing,plot


# 1. Leer, anotar y crear gráficas

In [ ]:
record = wfdb.rdrecord("ptb-diagnostic-ecg-database-1.0.0/patient001/s0010_re")

Utilizamos el comando para obtener la señal almacenada en el objeto record

In [ ]:
data = record.p_signal.T

Utilizamos el comando sig_name para obtener los nombres de las señales almacenadas en el objeto record

In [ ]:
name = record.sig_name

El atributo fs nos da información de la frecuencia de muestreo de la señal

In [ ]:
freq = record.fs

Comprobamos las unidades de las señales.

In [ ]:
unit = record.units

Vamos a jugar con la señal, antes de crear el gráfico con las funciones de wfdb. Creamos el gráfico para una única derivación, ajustando el tiempo en segundos y la intensidad en mV. A partir de este generaremos un gráfico más realista con las 15 derivaciones.

In [ ]:
num = 0 #cambiando el número podemos cambiar de derivación

ps.line(y=data[num],x=np.arange(0,38400)/freq, labels={'x':'t (s)', 'y':unit[num]},title="Signal Name " + name[num])

In [ ]:

wfdb.plot_wfdb(record,title="prueba",figsize=(40,30),ecg_grids="all")

Comprobamos que la función para generar electrocardiogramas funciona correctamente. Vemos varias partes a mejorar si un especialista quisiera darle  un uso convencional al programa: no es necesario obtener todo el rango de la señal y las últimas derivaciones no suelen usarse tanto si queremos obtener información básica. A modo de práctica vamos a coger una ventana dentro del electro y eliminar las tres últimas derivaciones

In [ ]:
brief_record = wfdb.rdrecord('ptb-diagnostic-ecg-database-1.0.0/patient001/s0010_re',sampfrom=1000, sampto=6000,channel_names=['i',
 'ii',
 'iii',
 'avr',
 'avl',
 'avf',
 'v1',
 'v2',
 'v3',
 'v4',
 'v5',
 'v6'])

brief_annotation = wfdb.rdann('ptb-diagnostic-ecg-database-1.0.0/patient001/s0010_re', 'xyz',sampfrom=1000 ,sampto=6000)

In [ ]:
wfdb.plot_wfdb(record=brief_record, annotation=brief_annotation, plot_sym=True,
                   time_units='seconds', title='MIT-BIH Record 100',
                   figsize=(40,30), ecg_grids='all')

Si queremos detectar y medir la frecuencia cardiaca, hay variaos algoritmos publicados, muchos de ellos ya integrados en librerias  de programación para facilitar su uso. Yo he didido usar GQRS dentro de wfdb.

In [ ]:
record_2 = wfdb.rdrecord('ptb-diagnostic-ecg-database-1.0.0/patient001/s0010_re', channels=[0], physical=False)

qrs_locs_2 = processing.gqrs_detect(d_sig=record_2.d_signal[:,0],
                                        fs=record_2.fs,
                                        adc_gain=record_2.adc_gain[0],
                                        adc_zero=record_2.adc_zero[0])

Una vez calculado el complejo QRS, es facil detectar la frecuencia cardiaca, debemos calcular el tiempo entre complejos. Aunque hay una función dedicada, la parte teórica sería saber cuanto tiempo ha pasado entre un complejo y otro (Q(t)-Q(t-1)) lo mismo para R o S.

In [ ]:
intervals = processing.calc_rr(qrs_locs_2, fs=record_2.fs)
print(intervals)


Una vez calculados los intervalos entre complejos, lanzamos la función para calcular el ritmo cardiaco medio dentro del electro.

In [ ]:
hrm= processing.calc_mean_hr(intervals,fs=record_2.fs)

print("El ritmo cardiaco medio es de {} pulsaciones por minuto".format(hrm))

Conociendo el complejo QRS es muy facil calcular cualquier otra propiedad relacionada con el ritmo cardiaco. En el siguiente ejemplo calculamos el ritmo cardiaco entre cada complejo.

In [ ]:

wfdb.processing.compute_hr(record_2.sig_len, qrs_locs_2, fs=record_2.fs)




Una vez conseguido el complejo qrs podemos guardarlo como anotación o bien crear una gráfica fusionando señal y anotación.

In [ ]:
processing.rr2ann(intervals,"ptb-diagnostic-ecg-database-1.0.0/patient001/anotation_rr_test","atr",record_2.fs)

Mostrar las anotaciones sobre el gráfico también es una buena práctica para comprobar que ha funcionado el algoritmo de detección de complejo QRS

In [ ]:
plot.plot_items(record_2.d_signal/record_2.fs,[qrs_locs_2],figsize=(40,10),sig_units=record_2.units,ecg_grids=[0],fs=record_2.fs,time_units="seconds")

Creo que hay un fallo en la función anterior que no utiliza correctamente las unidades propuestas, provocando que aparezcan infinitas líneas (grid) en la gráfica. Para solucionarlo he divido la señal por 1000 para transformarla en mV, obteniendo un resultado positivo.

# 2. Parte de desarrollo libre.
Teniendo en cuenta que cada paciente cuenta con un electro de 15 derivaciones, se puede sacar partido  de esto y plantearlo como información 2D. Al final una señal 2D no deja de ser (en cierta medida) parecido a una imagen, por lo que podemos aplicar algoritmos de deep learning para resolver nuestro problema.

El objetivo es montar una red de clasificación que puede detectar de forma precisa entre pacientes con Myocardial infarction y pacientes sanos. La red que voy a implementar es muy simple para demostrar que es posible tratar una señal de ecg como una imagen. Esto se podria mejorar mucho más aplicando convoluciones y otro tipo de arquitecturas.

In [ ]:
import scipy
import keras
import tensorflow as tf
import glob
import os 

Genero la base de datos, leyendo la información de cada paciente y guardando su ecg en una carpeta nombrada con la patología. 

In [ ]:

root = "ptb-diagnostic-ecg-database-1.0.0"
paths = glob.glob(root+"/patient*/*dat")
for path in paths:
  path_splited = path.split("/")
  patient_num = path_splited[1]
  file_name = path_splited[2]
  record = wfdb.rdrecord(path.split(".dat")[0])
  reason_admission = record.comments[4].split("Reason for admission: ")[1]
  if not os.path.exists(reason_admission):
    os.makedirs(reason_admission) 
  else:
    np.save(reason_admission+"/"+patient_num+"_"+file_name.split(".")[0]+".npy",record.p_signal.T[:,:32000])

In [ ]:
signals=[]

for file in os.listdir("Myocardial infarction/"):
  signals.append(np.load("Myocardial infarction/"+file))

labels_infarction=np.ones(len(signals))

for file in os.listdir("Healthy control/"):
  signals.append(np.load("Healthy control/"+file))

labels_controls=np.ones(len(signals)-labels_infarction.shape[0])*0

labels = np.append(labels_infarction,labels_controls)
signals = np.dstack(signals)
signals = np.rollaxis(signals,-1)
print(signals.shape,labels.shape)
train_dataset = tf.data.Dataset.from_tensor_slices((signals, labels))



In [ ]:
train_dataset.save()

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((signals, labels))


Ahora devemos tomar una parte para validation, train y test. Para ello utilizo una función ya implementada en tf

In [ ]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

Aleatorizo el dataset y creo el batch de entrenamiento

In [ ]:
 train_ds, val_ds, test_ds=get_dataset_partitions_tf(train_dataset,labels.shape[0])

In [ ]:
BATCH_SIZE = 1
SHUFFLE_BUFFER_SIZE = 100

val_ds = val_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


In [ ]:
BATCH_SIZE = 10
SHUFFLE_BUFFER_SIZE = 100

train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

Con las siguientes líneas de código almacenamos memoria en cache para ahorrar espacio en ram.



In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
BATCH_SIZE = 1
SHUFFLE_BUFFER_SIZE = 100

test_ds = test_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
print(train_ds, val_ds,test_ds)

Genero una red densamente conexa muy sencilla para clasificar

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(15, 32000)),
   
        tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(1,activation="sigmoid")
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])


In [ ]:
model.summary()

Entrenamos la red con el dataset generado

In [ ]:
history = model.fit(train_ds,validation_data=val_ds, batch_size=30, epochs=20)

In [ ]:
model.evaluate(test_ds)


## Los resultados finales para el conjunto de test (46 ecgs) son un 91.30% de accuracy y una perdida de 7.54.

Estos resultados se pueden mejorar en gran medida aplicando redes convolucionales e incluso añadiendo capas densas a la arquictura original.

Esto demuestra que los algoritmos de Deep Learning pueden ser usados en clasificación de ecg.


# Continuación ejercicio libre aplicando CNN

Siguiendo con las líneas futuras planteadas anteriormente, diseñamos una arquitectura CNN. 

Creamos una capa específica RESNET (https://arxiv.org/abs/1512.03385v1) que potencia el gradiente. Mantenemos los parámetros de entrenamiento. Aplicando convoluciones hemos reducido el número de parámetros de la red de 61 millones a 7 millones.

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters,name):
    super(ResnetIdentityBlock, self).__init__(name=name)
    filters1, filters2, filters3 = filters
    
    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    y = tf.nn.relu(x)

    x = self.conv2b(y)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += y
    return tf.nn.relu(x)


In [ ]:
input = tf.keras.Input((15, 32000,1))

c1 = ResnetIdentityBlock(filters=[16,16,16],kernel_size=(3,3),name="resent1")(input)

m1 = tf.keras.layers.MaxPool2D((1,5))(c1)

c2 =  ResnetIdentityBlock(filters=[32,32,32],kernel_size=(3,3),name="resent2")(m1)

m2 = tf.keras.layers.MaxPool2D((1,5))(c2)

c3 =  ResnetIdentityBlock(filters=[32,32,32],kernel_size=(3,3),name="resent3")(m2)

m3 = tf.keras.layers.MaxPool2D((1,5))(c3)
f1= tf.keras.layers.Flatten()(m3)
d1 = tf.keras.layers.Dense(64, activation='relu')(f1)

d2= tf.keras.layers.Dense(1,activation="sigmoid")(d1)
model=tf.keras.Model(input,d2)

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

Entrenamos la red con el dataset generado

In [ ]:
history = model.fit(train_ds,validation_data=val_ds, batch_size=30, epochs=20)

Hemos conseguido mejorar los resultados de la red, no solo mejorando el accuracy sino también hemos reducido el loss un orden de magnitud.

In [ ]:
model.evaluate(test_ds)


IMPORTANTE: Para llevar a cabo buenas prácticas se debería de realizar un cross-validation para cada uno de los entrenamientos.

# ¿Cómo pienso que se podría mejorar?

El ejemplo anterior es solo una idea de lo que podría ofrecer a Idoven como Data Scientist. A continuación, propongo mejoras que podrían hacerse al problema de clasificación. 

Añado que al igual que hemos clasificado entre pacientes sanos y con Myocardial infarction, también se podrían añadir el resto de clases por las que un paciente es ingresado (Palpitation, Myocarditis...).

El bonito mundo de las convoluciones. Se ha demostrado en varios artículos que en muchos casos (donde hay información espacial sobre todo) mejoran al comportamiento de las redes densas (el ejemplo anterior) y necesitan menor computo.

Teniendo esto en cuenta es dificil aplicar convoluciones a un ecg de la forma que se presentaba anterior mente (una matriz de 15X32000). Para concentrar la información podemos usar un espectrograma que através de diferentes  transformaciones en frecuencia con la señal, se obtine una imagen.

In [ ]:
_,_,a=scipy.signal.spectrogram(record_2.d_signal.T,fs=record_2.fs)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(a[0][::-1,:])
plt.tight_layout()

Una vez obtenida esta imagen, podemos aplicar un logaritmo para potenciar las partes con información. De igual manera, una vez concentrada la información podemos aplicar convoluciones 2D y volver a montar una red de clasificación, añadiendo si quisieramos el resto de etiquetas.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(np.log(a[0])[::-1,:])
plt.tight_layout()